In [1]:
# !pip install transformers
# !pip install -U sentence-transformers
!pip install wordsegment
!pip install pyvi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.8 MB 36.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.5 MB 35.6 MB/s 
     |████████████████████████████████| 965 kB 62.8 MB/s 


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
print(tokenizer("hôm nay Tôi Đi hỌc"))

In [2]:
from google.colab import drive
drive.mount('/content/drive')
# sau khi run xong bước này, vào biểu tượng file bên phải -> vào drive/MyDrive/Colab Notebooks, tạo folder tên dataset -> upload 2 file dataset vào thư mục này

Mounted at /content/drive


In [3]:
import pandas as pd
import string

# Spacy for english word segmentation
import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
# Pyvi for vietnam word segmentation
from pyvi import ViTokenizer

# import pandas as pd
import os
import io
import string
import re

In [4]:
# !ls "/content/drive/MyDrive/Colab Notebooks/dataset/en_sents"
dataset_path = "/content/drive/MyDrive/Colab Notebooks/dataset/"
spacy_en = spacy.load('en_core_web_sm')

eos = '<eos>'
bos = '<bos>'

#  scr - tar - model_eng-viet save
#  scr - tar - model_viet-eng save
# ?train bert - phobert
# def create_raw_dataset():
#     data_dir = "/content/drive/MyDrive/Colab Notebooks/dataset/"
#     en_sents = open(data_dir + 'en_sents', "r").read().splitlines()
#     vi_sents = open(data_dir + 'vi_sents', "r").read().splitlines()
#     return {
#         "English": [line for line in en_sents[:5000]],
#         "Vietnamese": [line for line in vi_sents[:5000]],
#     }
# raw_data = create_raw_dataset()
# df = pd.DataFrame(raw_data, columns=["English", "Vietnamese"])
# print(df)

In [ ]:
def tokenize_en(en_dtset):
  en_return = []
  for sent in en_dtset:
    en_return.append([tok.text for tok in spacy_en.tokenizer(sent)])

  # add begin and end to sentence
  for i in range(len(en_return)):
    en_return[i].insert(0, bos)
    en_return[i].insert(len(en_return[i]), eos)
  return en_return

def tokenize_vi(vi_dtset):
  vi_return = []
  for sent in vi_dtset:
    vi_return.append(ViTokenizer.tokenize(sent).split())

  # add begin and end to sentence
  for i in range(len(vi_return)):
    vi_return[i].insert(0, bos)
    vi_return[i].insert(len(vi_return[i]), eos)
  return vi_return

en_sentences = open(dataset_path + 'en_sents', 'r').read().splitlines()
en_lst = [line.lower() for line in en_sentences[:10000]]

vi_sentences = open(dataset_path + 'vi_sents', 'r').read().splitlines()
vi_lst = [line.lower() for line in vi_sentences[:10000]]

en_seg = tokenize_en(en_lst)
# print(en_seg)

vi_seg = tokenize_vi(vi_lst)
vi_seg


In [18]:
# Word embedding
from gensim.models import Word2Vec

vi_model = Word2Vec(vi_seg, size=100, window=2, min_count=1, sample=0.0001, workers=4, sg=0, negative=10, cbow_mean=1, iter=5)

class Language():
    def __init__(self, lines):
        self.lines = lines
        self.word2id = {}
        self.id2word = {}
        self.vocab = set()
        self.max_len = 0
        self.min_len = 0
        self.vocab_size = 0
        self.init_language_params()

    def init_language_params(self):
        self.word2id['<pad>'] = 0
        for line in self.lines:
            self.vocab.update(line)
        for id, word in enumerate(self.vocab):
            self.word2id[word] = id + 1
        for word, id in self.word2id.items():
            self.id2word[id] = word
        
        self.max_len = max([len(line) for line in self.lines])
        self.min_len = min([len(line) for line in self.lines])
        self.vocab_size = len(self.vocab) + 1


scr_en = Language(en_seg)
tar_vi = Language(vi_seg)

tar_vi.vocab

{'boston',
 'gõ_cửa',
 'thơ',
 'cảm_ơn',
 'vài',
 'mức_độ',
 'bận_tâm',
 'chấn_thương',
 'hình',
 'mù',
 'dịch_thuật',
 'vì',
 'tín_dụng',
 'trang_bị',
 'á',
 'susan',
 'uống',
 'cân',
 'thẳng',
 'niềm',
 'chịu_thua',
 'còi',
 'rưỡi',
 'tulip',
 'màn',
 'telegram',
 'vật_tư',
 'disneyland',
 'co_rúm',
 'mary',
 'vững_chắc',
 'chữa_trị',
 'hợp_tác',
 'rõ_ràng',
 'vương_quốc',
 'hèn_nhát',
 'b',
 'bọ_cạp',
 'malaysia',
 'mòn',
 'cốc',
 'hoan_nghênh',
 'cuộn',
 'làm_việc',
 'bóng_rổ',
 'con',
 'bao_tải',
 'đứng',
 'rẻ',
 'thận_trọng',
 'ôn_hòa',
 'nổi',
 'đảm_nhận',
 'nghe',
 'đáp',
 'chết_mệt',
 'gửi',
 'nhạy_cảm',
 'hình_ảnh',
 '1903',
 'dự_định',
 'khiêm_tốn',
 'biết_bao_nhiêu',
 'nancy',
 'thực_hành',
 'chín',
 'scottish',
 'quỳ',
 'bê',
 'đó',
 'chính_trị',
 'tất',
 'ống',
 'hoàn_thành',
 'ra_lệnh',
 'york',
 'chảy_máu',
 'trai_trẻ',
 'dna',
 '13',
 'bồn_tắm',
 'vào_khoảng',
 'cà_vạt',
 '<bos>',
 'truyền_nhiễm',
 'có_học',
 'công_việc',
 'săn_lùng',
 'cá_mập',
 'video',
 'bão',
 'thự

{'go', 'i'}

In [ ]:
# pip install transformers && pip install torch

In [ ]:
# from transformers import DistilBertTokenizerFast, DistilBertModel

# tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
# tokens = tokenizer.encode('This is a input.', return_tensors='pt')
# print("These are tokens!", tokens)
# for token in tokens[0]:
#     print("This are decoded tokens!", tokenizer.decode([token]))

In [ ]:
# model = DistilBertModel.from_pretrained("distilbert-base-uncased")
# print(model.embeddings.word_embeddings(tokens))
# for e in model.embeddings.word_embeddings(tokens)[0]:
#     print("This is an embedding!", e)

In [ ]:
# from sentence_transformers import SentenceTransformer, models

# ## Step 1: use an existing language model
# word_embedding_model = models.Transformer('distilroberta-base')

# ## Step 2: use a pool function over the token embeddings
# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
# # print('pool: ', pooling_model)

# ## Join steps 1 and 2 using the modules argument
# model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
# print('model: ', model)


Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model:  SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)
